# IETF Affiliations from Attendance Records

In [72]:
import bigbang.domain as domain

from ietfdata.datatracker     import *
from ietfdata.datatracker_ext import *
import pandas as pd
import matplotlib.pyplot as plt
import dataclasses

## Getting generic email domains

Will use these later...

In [73]:
!wget https://gist.githubusercontent.com/ammarshah/f5c2624d767f91a7cbdc4e54db8dd0bf/raw/660fd949eba09c0b86574d9d3aa0f2137161fc7c/all_email_provider_domains.txt

--2021-11-30 13:44:00--  https://gist.githubusercontent.com/ammarshah/f5c2624d767f91a7cbdc4e54db8dd0bf/raw/660fd949eba09c0b86574d9d3aa0f2137161fc7c/all_email_provider_domains.txt
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 85116 (83K) [text/plain]
Saving to: ‘all_email_provider_domains.txt.2’

all_email_provider_ 100%[===================>]  83.12K   276KB/s    in 0.3s    

2021-11-30 13:44:00 (276 KB/s) - ‘all_email_provider_domains.txt.2’ saved [85116/85116]



In [78]:
aepd = open("all_email_provider_domains.txt").read().split("\n")

## Getting attendence records from datatracker

When attendees register for a meeting, the report their name, email address, and affiliation.

While this is noisy data (any human-entered data is!), we will use this information to associate domains with affilations. E.g. the email domain `apple.com` is associated with the company Apple.

We will also use this data to enrich our understanding of individual affiliations over time.

In [2]:
datatracker = DataTracker()

meetings = datatracker.meetings(meeting_type = datatracker.meeting_type(MeetingTypeURI('/api/v1/name/meetingtypename/ietf/')))
full_ietf_meetings = list(meetings)

INFO:ietfdata:glasgow-ietfdata/0.5.1 (cache disabled)


In [3]:
ietf_meetings = []
for meeting in full_ietf_meetings:
    meetingd = dataclasses.asdict(meeting)
    meetingd['meeting_obj'] = meeting
    meetingd['num'] = int(meeting.number)
    ietf_meetings.append(meetingd)    

In [4]:
meetings_df = pd.DataFrame.from_records(ietf_meetings)

## Individual Affiliations

In [7]:
dt = DataTrackerExt() # initialize, for all meeting registration downloads

INFO:ietfdata:glasgow-ietfdata/0.5.1 (cache disabled)


This will construct a dataframe of every attendee's registration at every specified meeting. (Downloading this data takes a while!)

In [12]:
ietf_meetings[110]['date']

datetime.datetime(2021, 7, 24, 0, 0)

In [108]:
meeting_attendees_df = pd.DataFrame()
for meeting in ietf_meetings:
    if meeting['num'] in [104,105,106,107,108,109]: # can filter here by the meetings to analyze
        registrations = dt.meeting_registrations(meeting=meeting['meeting_obj'])
        df = pd.DataFrame.from_records([dataclasses.asdict(x) for x in list(registrations)])
        df['num'] = meeting['num']
        df['date'] = meeting['date']
        df['domain'] = df['email'].apply(domain.extract_domain)
        full_name = df['first_name'] + " " + df['last_name']
        df['full_name'] = full_name
        meeting_attendees_df = meeting_attendees_df.append(df)

Filter by those who actually attended the meeting (checked in, didn't just register).

In [109]:
ind_affiliation = meeting_attendees_df[['full_name', 'affiliation', 'email', 'domain','date']]

This format of data -- with name, email, affiliation, and a timestamp -- can also be extracted from other IETF data, such as the RFC submission metadata. Later, we will use data of this form to infer _duration_ of affilation for IETF attendees.

In [110]:
ind_affiliation[:10]

,full_name,affiliation,email,domain,date
0,Thomas Pauly,Apple,tpauly@apple.com,apple.com,2019-03-23
1,Eric Kinnear,Apple,ekinnear@apple.com,apple.com,2019-03-23
2,Jordi Palet Martinez,Moremar,jordi.palet@consulintel.es,consulintel.es,2019-03-23
3,Heather Flanagan,RFC Editor,rse@rfc-editor.org,rfc-editor.org,2019-03-23
4,Kyle Rose,Akamai Technologies,krose@krose.org,krose.org,2019-03-23
5,Aaron Falk,Akamai,aaron.falk@gmail.com,gmail.com,2019-03-23
6,Russ Housley,"Vigil Security, LLC",housley@vigilsec.com,vigilsec.com,2019-03-23
7,Jason Livingood,Comcast // IASA 2.0 WG,Jason_Livingood@comcast.com,comcast.com,2019-03-23
8,Jeff Osborn,Internet Systems Consortium,jeff@isc.org,isc.org,2019-03-23
9,Mahesh Jethanandani,VMware,mjethanandani@gmail.com,gmail.com,2019-03-23


## Matching affiliations with domains

In [111]:
affil_domain = ind_affiliation[['affiliation', 'domain', 'email']].pivot_table(
    index='affiliation',columns='domain', values='email', aggfunc = 'count')

In [112]:
generic_email_domains = set(affil_domain.columns).intersection(set(aepd))

In [113]:
affil_domain.drop(generic_email_domains, axis = 1, inplace = True)

In [116]:
ad_max = affil_domain.apply(lambda row: row.max(), axis=1)
ad_mean = affil_domain.apply(lambda row: row.dropna().mean(), axis=1)
ad_count = affil_domain.apply(lambda row: row.dropna().count(), axis=1)

ad_max_domain = affil_domain.apply(lambda row: row.idxmax(), axis=1)

## Add the columns *after* computing the statistics!
affil_domain['max'] = ad_max
affil_domain['mean'] = ad_mean
affil_domain['count'] = ad_count
affil_domain['max_domain'] = ad_max_domain

In [117]:
ad_stats = affil_domain[['max_domain','max','count','mean']].sort_values('max', ascending=False)

In [118]:
ad_stats[:100]

domain,max_domain,max,count,mean
affiliation,,,,
Huawei,huawei.com,132.0,4,33.750000
Cisco,cisco.com,126.0,6,23.166667
Cisco Systems,cisco.com,124.0,3,42.666667
Ericsson,ericsson.com,103.0,8,15.625000
Juniper Networks,juniper.net,97.0,2,49.500000
...,...,...,...,...
Telefonica,telefonica.com,7.0,1,7.000000
TU Berlin,inet.tu-berlin.de,7.0,3,3.000000
Broadcom,broadcom.com,7.0,1,7.000000


In [119]:
ad_stats[:100].to_csv("affiliation_domain_stats.csv")

## Duration of affiliation

The current data we have for individual affiliations is "point" data, reflecting the affiliation of an individual on a particular date.

For many kinds of analysis, we may want to understand the full duration for which an individual has been associated with an organization. This requires an inference from the available data points to dates that are not explicitly represented in the data.

For now, we will use a rather simple form of inference: filling in any missing data from the last (temporally) known data point. And then if there's still missing data, infer backwards.

In [134]:
affil_dates = ind_affiliation.pivot_table(
    index="date",
    columns="full_name",
    values="affiliation",
    aggfunc="first"
).fillna(method='ffill').fillna(method='bfill')

In [147]:
top_attendees = ind_affiliation.groupby('full_name')['date'].count().sort_values(ascending=False)[:40].index

In [148]:
top_attendees

Index(['Ignas Bagdonas', 'Martin Duke', 'Gert Grammel', 'Roni Even',
       'Linda Dunbar', 'Toerless Eckert', 'Richard Barnes', 'Kohei Isobe',
       'Yutaka OIWA', 'Jonathan Lennox', 'Jim Reid', 'Ronald in 't Velt',
       'Gonzalo Camarillo', 'Paul Ebersman', 'Martin Thomson',
       'Marten Seemann', 'Glenn Deen', 'Martin Vigoureux', 'Paul Congdon',
       'Tianran Zhou', 'Ramesh Sivakolundu', 'Tero Kivinen', 'Markus Amend',
       'Ted Hardie', 'Chris Bowers', 'Tal Mizrahi', 'Takuya Miyasaka',
       'Chonggang Wang', 'Dominique Lazanski', 'Gorry Fairhurst',
       'Dino Farinacci', 'Tadahiko Ito', 'Suzanne Woolf', 'Susan Hares',
       'Suresh Krishnan', 'Matthew Ford', 'Dieter Sibold', 'Mark Nottingham',
       'Paul Hoffman', 'Marcus Ihlar'],
      dtype='object', name='full_name')

In [149]:
affil_dates[top_attendees]

full_name,Ignas Bagdonas,Martin Duke,Gert Grammel,Roni Even,Linda Dunbar,Toerless Eckert,Richard Barnes,Kohei Isobe,Yutaka OIWA,Jonathan Lennox,...,Dino Farinacci,Tadahiko Ito,Suzanne Woolf,Susan Hares,Suresh Krishnan,Matthew Ford,Dieter Sibold,Mark Nottingham,Paul Hoffman,Marcus Ihlar
date,,,,,,,,,,,,,,,,,,,,,
2019-03-23,Equinix,"F5 Networks, Inc.",Juniper,Huawei Technologies,Futurewei,Huawei USA,Cisco,SECOM,,Vidyo,...,,,,,Kaloom,Internet Society,PTB,,ICANN,Ericsson
2019-07-20,Equinix,"F5 Networks, Inc.",Juniper Networks,Toga Networks,Futurewei,Futurewei Technologies USA,Cisco,SECOM,AIST Japan / 産業技術総合研究所,8x8,...,lispers.net,SECOM,Public Interest Registry (.org),,Kaloom,Internet Society (ISOC),PTB,Fastly,ICANN,Ericsson
2019-11-16,Equinix,"F5 Networks, Inc.",Juniper,Toga Networks,Futurewei,Futurewei USA,Cisco,SECOM,AIST Japan,8x8 / Jitsi,...,,SECOM,Public Interest Registry (.org),,Kaloom,Internet Society,PTB,Fastly,ICANN,Ericsson
2020-03-21,Equinix,F5 Networks,Juniper,Toga Networks,,Futurewei USA,Cisco,SECOM,AIST Japan,8x8 / Jitsi,...,lispers.net,,Public Interest Registry (.org),,Kaloom,Internet Society,,Fastly,ICANN,Ericsson
2020-07-25,Equinix,"F5 Networks, Inc.",Juniper,,Futurewei,Futurewei USA,Cisco,SECOM,AIST Japan,,...,lispers.net,"SECOM CO., LTD.",Public Interest Registry (.ORG),Huawei,Kaloom,Internet Society,PTB,Fastly,ICANN,Ericsson
2020-11-14,Equinix,"F5 Networks, Inc.",Juniper,,Futurewei,Futurewei USA,Cisco,SECOM,AIST Japan,8x8 / Jitsi,...,lispers.net,SECOM,Public Interest Registry (PIR),Hickory Hill Consulting,Kaloom,Internet Society (ISOC),PTB,Fastly,ICANN,Ericsson


In [150]:
affil_dates[top_attendees].to_csv("inferred_affiliation_dates.csv")